## Prep MHFL for input into the processing script

In [3]:
import numpy as np
import pandas as pd
import uuid
import collections
import os
import re
import unidecode
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
from ordered_set import OrderedSet
pd.set_option('mode.chained_assignment', None)

import arcpy
from arcpy import env
from arcgis.features import GeoAccessor, GeoSeriesAccessor, SpatialDataFrame

C:\Users\hengin\AppData\Local\ESRI\conda\envs\arcgispro-py3-clone\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Initialize variables

In [14]:
# data import

# import dataset as df
dataDir = r"D:\Grid3\ISS\inputs\HF Countries data.gdb"
country_code = "CMR" # This is used for writing files out
country_prefix = "CMR" # This is used for input file name
mfl = "Formation_sanitaire_PEV_OMS_"#""MLHF_" + country_prefix
mfl_file = os.path.join(dataDir, mfl)

COUNTRY = 'country'
#COUNTRY_NAME = 'Democratic Republic of the Congo'
COUNTRY_NAME = 'Cameroon'

# current fields in mlhf
full_hf_name = "health_fac"
hf_type = "type"
admin1="region"
admin2="district_n"
admin3="area_name"
# new fields 
# full_clean_name = "facility_clean"
# short_name = "facility_short"
# match_type = "type_clean"

# hf_match_code = "HF_Code"

# output columns
FACILITY_NAME = "health_fac"
CLEAN_NAME = "facility_clean" # clean name after some pre-cleaning
CORRECT_NAME = 'corrected_name' # clean name after spelling correction
CLEAN_NAME_FINAL = "facility_short" # final clean name after removing type information
EXTRACT_TYPE = 'type_clean' # type information extracted
SUB_TYPE = "type_clean" # type mapped to the type dictonary
SCORE = 'score' # match score between 'type_extract' and 'sub_type'
mfl_uuid="mfl_uuid"
hf_match_code = "HF_Code"


# import type dictionary as TYPE_DICT
path_to_type_dict = r"D:\Grid3\ISS\inputs\spelling_type_dict\cmr_type_dict_augmented_1130.csv"
TYPE_DICT = pd.read_csv(path_to_type_dict )

# import spelling dictionary as SPELLING_DICT
path_to_spelling_dict = r"D:\Grid3\ISS\inputs\spelling_type_dict\cmr_spelling_dict_052021.csv"
SPELLING_DICT = pd.read_csv(path_to_spelling_dict )

#final output
SAVE_PATH =r"D:\Grid3\ISS\processing\MFL_by_country\mfl_by_country.gdb"
OUT_FILE = country_code + "_mfl"
SAVE_FILE = os.path.join(SAVE_PATH, OUT_FILE)
SAVE_FILE_csv=os.path.join("\\".join(SAVE_PATH.split("\\")[:-1]), OUT_FILE, ".csv")

## Functions

In [5]:
def preclean(df, facility_name, clean_name):
    
    # replace NAs with empty string ''
    df[facility_name] = df[facility_name].fillna('')
     # remove accent marks
    df[clean_name] = [unidecode.unidecode(n) for n in df[facility_name]]
    df[clean_name]=df[clean_name]+" "
    df[clean_name] = df[clean_name].str.replace(" III "," 3 ")\
        .str.replace(" II "," 2 ")\
        .str.replace(" I "," 1 ")\
        .str.replace(" Iii "," 3 ")\
        .str.replace(" Ii "," 2 ")\
        .str.replace(" IV "," 4 ")\
        .str.replace(" Iv "," 4 ")
    df[clean_name]  =df[clean_name].apply(lambda x: " ".join(re.split('(\d+)', x)))
    df[clean_name] = df[clean_name].str.strip()\
            .str.title()\
            .str.replace("  ", " ")\
            .str.replace('.', ' ')\
            .str.replace(':', ' ')\
            .str.replace("'", ' ')\
            .str.replace('"', ' ')\
            .str.replace('[', ' ')\
            .str.replace(']', ' ')\
            .str.replace('[-_,/\(\);.]', ' ')\
            .str.replace('&', ' and ')\
            .str.replace("  ", " ")\
            .str.strip()\
            .str.replace('center ', 'centre ', case=False)\
            .str.replace('^st ', ' saint ', case=False)\
            .str.replace('cl ', 'clinique ', case=False)\
            .str.replace('Geral ', 'General ', case=False)\
            .str.replace('Hospitals ', 'Hopital ', case=False)\
            .str.replace('Hospital ', 'Hopital ', case=False)\
            .str.replace("Urban ", "Urbain ", case=False)\
            .str.replace("Distrital ", "District ", case=False)\
            .str.replace('^hosp | hosp | hosp$|^hosp$', ' Hopital ', case=False)\
            .str.replace("  ", " ")\
            .str.strip()\
            .str.replace(" De ", " de ")\

            #.str.replace('Clinique', 'Clinic', case=False)\
            #.str.replace('Polyclinique', 'Polyclinic', case=False)\
            #.str.replace('Dispensaire', 'Dispensary', case=False)\
            # .str.replace('Hôpital', 'Hospital', case=False)\
            #.str.replace('Hopital', 'Hospital', case=False)\
    
    # replace NAs in clean_name with empty string ''
    df[clean_name] = df[clean_name].fillna('')
    
    # change emptry string in facility_name back to NA
    df[facility_name] = df[facility_name].replace('', np.nan)

    
    
    return df

### `corrected_name`

Makes correction to possible misspellings using the spelling dictionary.

In [6]:
def correct_spelling(df, spelling_dict, country_col, clean_name, output_col):
    corrected_results = pd.DataFrame()

    for country_name in df[country_col].unique():
        # obtain dataset for the country
        df_ctr = df[df[country_col].str.upper()==country_name.upper()]
    
        spelling_dict_ctr = spelling_dict[spelling_dict['Country'].str.upper()==country_name.upper()]
        words_to_correct = spelling_dict_ctr['Word'].unique()

        df_ctr[output_col] = df_ctr[clean_name]
        for word in words_to_correct:
            misspellings = list(spelling_dict_ctr[(spelling_dict_ctr['Word']==word)]['Misspelling'])
            for misspelling in misspellings:
                df_ctr[output_col] = df_ctr[output_col]\
                .str.replace('|'.join(['^'+misspelling+' ', ' '+misspelling+' ',
                                           ' '+misspelling+'$', '^'+misspelling+'$']), ' '+word+' ', case=False)\
                .str.strip().replace(" De "," de ")
    
        # merge country results to all results
        corrected_results = pd.concat([corrected_results, df_ctr])
    # reset and drop index
    corrected_results.reset_index(inplace=True, drop=True)
                                                              
    return corrected_results

In [7]:
def create_hf_code(df,clean_name, admin1, admin2,mfl_uuid):
    
    # replace NAs with empty string ''
    df["temp_admin"] = df[admin2].fillna('')
     # remove accent marks
    df["temp_admin"] = [unidecode.unidecode(n) for n in df["temp_admin"]]
    df["temp_admin"]=df["temp_admin"]+" "
    df["temp_admin"] = df["temp_admin"].str.replace(" III "," 3 ")\
        .str.replace(" II "," 2 ")\
        .str.replace(" I "," 1 ")\
        .str.replace(" Iii "," 3 ")\
        .str.replace(" Ii "," 2 ")\
        .str.replace(" IV "," 4 ")\
        .str.replace(" Iv "," 4 ")
    df["temp_admin"]  =df["temp_admin"].apply(lambda x: " ".join(re.split('(\d+)', x)))
    df["temp_admin"] = df["temp_admin"].str.title()\
            .str.replace('[-_,/\(\);.]', ' ')\
            .str.replace('  ', ' ')\
            .str.strip()

    # replace empty string '' with "NA"
    df["temp_admin"] = df["temp_admin"].fillna('NA')
    
    # create hf_code  by combining hf name and admin2 (district)
    df[hf_match_code] = df[clean_name]+"_"+df["temp_admin"]
    del df["temp_admin"]
    # create uuid for each facility
    def my_random_string(string_length=10):
        """Returns a random string of length string_length."""
        random = str(uuid.uuid4()) # Convert UUID format to a Python string.
        random = random.upper() # Make all characters uppercase.
        random = random.replace("-","") # Remove the UUID '-'.
        return random[0:string_length] # Return the random string.
    # create uuid 
    df[mfl_uuid]=""
    df["unique_name"]=df[clean_name]+"_"+df[admin1]+"_"+df[admin2]
    for name in df["unique_name"].unique():
        df.loc[df["unique_name"] == name, mfl_uuid] =my_random_string(string_length=10)
    del df["unique_name"]


In [8]:
def remove_type_info(df, type_dict, clean_name, clean_name_final, country_name):
    # remove whitespace between abbreviations of length 2 or 3
    # e.g. change C S to CS
    
    # convert country name to match country name in spelling and type dict
    df['country_eng']= country_name
    
    # obtain abbreviations of length 2 or 3
    tmp = type_dict[type_dict['Abbreviation'].str.len()<=3]['Abbreviation'].unique()
    # sort by decreasing length
    tmp = sorted(tmp, key=len, reverse=True)
    # change it to the pattern '^c s ' or ' c s$'
    tmp_dict = {}
    for t in tmp:
        tmp_dict[t] = ['^'+' '.join(list(t))+' ', ' '+' '.join(list(t))+'$']
    # replace the pattern with 'cs'
    for t in tmp:
        pats = tmp_dict[t]
        df[clean_name] = df[clean_name].str.replace(pats[0], t+' ',case=False)\
        .str.replace(pats[1], ' '+t, case=False)

    # remove type information
    df_grouped = df.groupby('country_eng')
    res = pd.DataFrame()

    for group_name, df_group in df_grouped:
        # obtain the type dictionary for that country
        tmp = type_dict[type_dict['Country'].str.upper()==group_name.upper()]

        # facility types for that country
        types = list(tmp['Type'])
        type_keywords = set()
        for t in types:
            # add the full facility type 
            t = t.title()
            type_keywords.add(t)                 

            # add individual words as well
            t = t.replace('/', ' ')
            words = t.split(' ')
            # skip words that have punctuation / numbers and have length <= 3 (e.g. de, (major))
            words = [w for w in words if w.isalpha() and len(w)>3]
            for w in words:
                type_keywords.add(w)

        # obtain the list of type keywords and sort in descending length
        type_keywords = list(type_keywords)
        type_keywords = sorted(type_keywords, key=lambda s: -len(s))

        # abbreviations for that country
        abbrevs = set(tmp['Abbreviation'])

        abb_keywords = []
        for abbrev in abbrevs:
            # e.g. for CS, 4 patterns are considered: '^CS ', ' CS ', ' CS$', '^CS$'
            abbrev = abbrev.title()
            abb_keywords.extend(['^'+abbrev+'\s', '\s'+abbrev+'\s', '\s'+abbrev+'$',
                                '^'+abbrev+'$'])

        # obtain the list of abbreviation keywords and sort in descending length
        abb_keywords = sorted(abb_keywords, key=lambda s: -len(s))  


        # handle situations when type is 'Hospital District' in the type dictionary 
        # but name column has 'District Hospital' in ISS data
        type_len_2 = [t for t in type_keywords if len(t.split())==2]
        for t in type_len_2:
            df_group[clean_name] = df_group[clean_name].str.title()\
            .str.replace(' '.join(t.split()[::-1]), t, case=False)

        # remove type information using keywords generated above
        # remove meaningless connecting words like de, do, da, du
        df_group[clean_name_final] = df_group[clean_name].str.title()\
            .str.replace('|'.join(type_keywords), '')\
            .str.replace('|'.join(abb_keywords), ' ')\
            .str.strip()\
            .str.replace('^de | de | de$|^de$|^do | do | do$|^do$|^da | da | da$|^da$|^du | du | du$|^du$', 
                         ' ', case=False)\
            .str.replace("  ", " ")\
            .str.strip()\
            .str.title()
        res = pd.concat([res, df_group])
    return res

In [9]:
def extract_type(df, clean_name, clean_name_final, extract_type):
    extract_types = []

    for idx, row in df.iterrows():
        name = row[clean_name].upper()
        name_final = row[clean_name_final].upper()

        # if clean_name_final is exactly the same as clean_name,
        # this indicates no type information can be extracted, thus append NA
        if name.upper() == name_final.upper():
            extract_types.append(np.nan)

        else:
            name = OrderedSet(name.split())
            name_final = OrderedSet(name_final.split())
            # find the difference between two names
            diff = ' '.join(list(name.difference(name_final)))
            extract_types.append(diff.strip())

    # remove de, do, da, du at start or end of extract_type
    # replace empty string with NA
    df[extract_type] = extract_types
    df[extract_type] = df[extract_type].str.strip()\
        .str.replace("  ", " ")\
        .str.replace('^de |^do |^da |^du | du$| de$| do$| da$|^de$|^do$|^da$|^du$', '', case=False)\
        .str.replace('^de |^do |^da |^du | du$| de$| do$| da$|^de$|^do$|^da$|^du$', '', case=False)\
        .str.strip()\
        .str.title()\
        .str.replace(" De ", " de ")\
        .replace('',"NA")
    df[clean_name]=df[clean_name].str.replace(' De ', " de ")
    df[clean_name_final].replace("", np.nan, inplace=True)
    df[clean_name_final].fillna("NA", inplace=True)
    df[extract_type].fillna("NA", inplace=True)

### `sub_type`

Use `extract_type` to map the type information extracted from the name column to one of the types in the type dictionary.

In [10]:
def map_type(df, country, extract_type, sub_type, score, type_dict):
    df_grouped = df.groupby(country)
    res = pd.DataFrame()
    for country_name in df[country].unique():
        df_group = df[df[country]==country_name]
        # obtain facility types and abbreviations for that country
        tmp = type_dict[type_dict['Country'].str.upper()==country_name.upper()]
        types, abbrevs = tmp['Type'], tmp['Abbreviation']
        sub_types = []
        scores = []

        for idx, row in df_group.iterrows():
            # if extract_type is NA, just append NA
            if not isinstance(row[extract_type],str):
                sub_types.append(np.nan)
                scores.append(np.nan)

            # find best match
            else:
                match, match_score = process.extractOne(row[extract_type], list(types)+list(abbrevs), scorer = fuzz.token_sort_ratio)
            
                scores.append(match_score)
                # if best match is abbreviation, map it to the corresponding type
                if match in list(abbrevs):
                    match_type = tmp[tmp['Abbreviation']==match]['Type'].iloc[0]
                    sub_types.append(match_type)
                else:
                    sub_types.append(match) 
        df_group[sub_type] = sub_types
        df_group[score] = scores
        res = pd.concat([res, df_group])
    return res

## Result

In [18]:
# read input dataset as spatial dataframe
sdf=pd.DataFrame.spatial.from_featureclass(mfl_file)
# pre-clean hf name
preclean(df=sdf, facility_name=full_hf_name, clean_name = CLEAN_NAME)

corrected_results = correct_spelling(sdf, spelling_dict=SPELLING_DICT, country_col=COUNTRY, 
                                     clean_name = CLEAN_NAME, output_col=CLEAN_NAME_FINAL)
# create hf code
create_hf_code(corrected_results,CLEAN_NAME_FINAL,admin1, admin2,mfl_uuid)
# # # separate type from name
clean_sdf = remove_type_info(df=corrected_results, type_dict=TYPE_DICT, clean_name=CLEAN_NAME, 
                             clean_name_final=CLEAN_NAME_FINAL,country_name=COUNTRY_NAME)
# # remove type from hf name
extract_type(df=clean_sdf, clean_name=CLEAN_NAME, 
              clean_name_final=CLEAN_NAME_FINAL, extract_type=EXTRACT_TYPE)

# final check for name and type

for index, row in clean_sdf.iterrows():
    if clean_sdf.at[index, EXTRACT_TYPE]=="NA":
        
        get_type=unidecode.unidecode(clean_sdf.at[index, hf_type])

        clean_sdf.at[index,EXTRACT_TYPE ]=get_type
    if clean_sdf.at[index,CLEAN_NAME_FINAL ]=="NA":
        clean_sdf.at[index,CLEAN_NAME_FINAL]=clean_sdf.at[index, EXTRACT_TYPE]

# check if facility duplicated
facility_uuid=clean_sdf[mfl_uuid].tolist()
dup_facility=[item for item, count in collections.Counter(facility_uuid).items() if count > 1]

clean_sdf["duplicate_uuid"]=0
for index, row in clean_sdf.iterrows():
    if clean_sdf.at[index, mfl_uuid ] in dup_facility :
        clean_sdf.at[index, "duplicate_uuid"]=1
  
##===================Result=========================##
print(">>> Number of facility in master facility list:")
print(clean_sdf.shape[0])
print()
print(">>> Summary by type of facilities:")
print(clean_sdf[EXTRACT_TYPE].value_counts())
print()

print(">>> Number of facility that duplicated in MFL:")
unique_facility=len(clean_sdf[mfl_uuid].unique())
print(clean_sdf.shape[0]-unique_facility)
print()
print(">>> Word frequency in facility name:")
clean_sdf[CLEAN_NAME_FINAL].str.split(expand=True).stack().value_counts()[:20]
# export result to spatial layer
clean_sdf["adm1_name"]=clean_sdf[admin1]
clean_sdf["adm2_name"]=clean_sdf[admin2]
clean_sdf["adm3_name"]=clean_sdf[admin3]
clean_sdf.spatial.to_featureclass(SAVE_FILE)
clean_sdf.to_csv(SAVE_FILE)

D66140B0-0277-45FF-AE18-A787BE3D4AAF:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
D66140B0-0277-45FF-AE18-A787BE3D4AAF:23: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
D66140B0-0277-45FF-AE18-A787BE3D4AAF:24: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
D66140B0-0277-45FF-AE18-A787BE3D4AAF:38: FutureWarning: The default value of regex will change from True to False in a future version.
D66140B0-0277-45FF-AE18-A787BE3D4AAF:17: FutureWarning: The default value of regex will change from True to False in a future version.
D661

>>> Number of facility in master facility list:
5458

>>> Summary by type of facilities:
Csi                   1624
Priv??                 772
Cs                     540
Ihc                    255
Cma                    219
                      ... 
Privee                   1
Hopital Catholique       1
Hospitalier              1
Hd de District           1
Regional Health          1
Name: type_extract, Length: 211, dtype: int64

>>> Number of facility that duplicated in MFL:
339

>>> Word frequency in facility name:


In [20]:


## get count of facility type
clean_sdf=pd.DataFrame.spatial.from_featureclass("CMR_mfl")
type_count=clean_sdf['facility_short'].value_counts()
type_count_df=pd.DataFrame(type_count).reset_index().rename({"type_clean":"frequancy", "index":"type"}, axis=1)
#type_count_df.to_csv("\\".join(SAVE_PATH.split("\\")[:-1])+"\\CMR_MFL_type_list.csv")

In [35]:
word_count=clean_sdf["facility_short"].str.split(expand=True).stack().value_counts()
word_count_df=pd.DataFrame(word_count).reset_index().rename({0:"frequancy", "index":"abrv"}, axis=1)
word_count_df=word_count_df[(word_count_df["abrv"].str.len()<=4)&(word_count_df["frequancy"]>=10)]